As células iniciais desse notebook se assemelham aos demais, que importam a base de dados e o arquivo de descrição para facilitar a identificação de cada variável pelo seu tipo.

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import pandas as pd
import yaml
import numpy as np
from numpy.typing import ArrayLike
import itertools

## Carregar conjunto de dados

In [ ]:
data = pd.read_csv('/content/drive/MyDrive/MedicinIA/Sample_Data/Processed/DataSet1.csv')
with open('/content/drive/MyDrive/MedicinIA/Sample_Data/Processed/DataSet1.yaml', 'r') as f:
    config = yaml.safe_load(f)

numerical = config['NumericalColumns']
categorical = config['CategoricalColumns']
identifier = config['IdentifierColumns']
datetime = config['DateTimeColumns']
target = config['Target']

## Agregação

**Criamos um conjunto para os dados numéricos, outros com os categóricos e todos eles foram agregados, contendo uma linha para cada paciente.**

### Numéricas

In [ ]:
agg_numerical = data[numerical+identifier].groupby('record_id').agg(
    bmi_median = ('bmi', 'median'),
    bmi_std = ('bmi', 'std'),
    bmi_min = ('bmi', 'min'),
    bmi_max = ('bmi', 'max'),
    sister_n = ('sister_n', 'max'),
    ki67_perc_mean = ('ki67_perc', 'mean'),
    follow_up_days = ('follow_up_days', 'mean'),
    follow_up_days_recidive = ('follow_up_days_recidive', 'mean'),
    follow_up_days_recidive_std = ('follow_up_days_recidive', 'std'),
    height = ('height', 'mean'), 
    weight_mean = ('weight', 'mean'),
    weight_std = ('weight', 'std')
)

In [ ]:
agg_numerical

,bmi_median,bmi_std,bmi_min,bmi_max,sister_n,ki67_perc_mean,follow_up_days,follow_up_days_recidive,follow_up_days_recidive_std,height,weight_mean,weight_std
record_id,,,,,,,,,,,,
54,29.15,0.435775,27.7,29.8,-1.0,19.6,3585.0,1338.0,0.0,152.100000,67.335000,1.014591
302,24.80,6.650833,22.4,45.0,-1.0,-1.0,2225.0,-1.0,0.0,157.909091,59.916000,2.921812
710,26.40,5.330967,0.0,45.0,-1.0,20.0,3294.0,2442.0,0.0,155.117647,63.451471,1.348497
752,35.90,0.525259,35.1,37.2,-1.0,-1.0,4153.0,-1.0,0.0,152.000000,83.110000,1.198923
1589,22.90,4.715171,22.2,45.0,-1.0,-1.0,3290.0,-1.0,0.0,167.000000,64.173684,1.305334
...,...,...,...,...,...,...,...,...,...,...,...,...
82057,38.80,0.843039,37.4,40.5,-1.0,30.0,441.0,-1.0,0.0,156.000000,94.550000,2.070887
82059,27.55,0.508060,26.8,28.6,-1.0,15.0,351.0,-1.0,0.0,164.000000,74.275000,1.346233
82122,25.25,0.240535,24.7,25.4,-1.0,-1.0,401.0,-1.0,0.0,155.000000,60.400000,0.568624


### Categóricas

No conjunto das variáveis categóricas, temos 70 colunas, estas foram distribuídas entre uni e multivaloradas. Para as univaloradas, agrupamos seus valores inserindo em cada linha o valor único respondido para aquele paciente. Às multivaloradas, por sua vez, agrupamos os múltiplos valores para aquele paciente em um só campo, separando-os por vírgula, como pode ser observado, por exemplo, no primeiro registro da coluna tumor_stage.  

In [ ]:
univalued_categories = [
    'abortion',
    'alcohol',
    'alcohol_type___1',
    'alcohol_type___2',
    'alcohol_type___3',
    'antec_fam_cancer_mama',
    'breast_feeding',
    'chemotherapy',
    'contraceptive_method___1',
    'contraceptive_method___2',
    'contraceptive_method___3',
    'contraceptive_method___4',
    'contraceptive_method___999',
    'diff_tubular',
    'er_ihc',
    'escolari',
    'familial_degree___1',
    'familial_degree___2',
    'familial_degree___3',
    'family_cancer',
    'family_cancer_id___1',
    'family_cancer_id___2',
    'family_cancer_id___3',
    'family_cancer_id___4',
    'family_cancer_id___5',
    'family_cancer_id___6',
    'grau_hist',
    'her2_fish',
    'her2_ihc',
    'histological_grade',
    'hormone_therapy',
    'mitotic_index',
    'nuclear_grade',
    'period',
    'pr_ihc',
    'pregnancy_history',
    'primary_diganosis',
    'race',
    'rec01',
    'rec02',
    'rec03',
    'rec04',
    'recdist',
    'recidive',
    'reclocal',
    'recregio',
    'tipotrat',
    'tobaco',
    'topo',
    'trathosp',
    'treatment']

In [ ]:
agg_operations = {x: pd.Series.mode for x in univalued_categories}
agg_operations.update(
    {x: 'unique' for x in set(categorical) - set(univalued_categories)}
)
agg_categorical = data.groupby('record_id').agg(agg_operations)

In [ ]:
multivalued_categories = categorical
for column in univalued_categories:
    multivalued_categories.remove(column)

In [ ]:
def post_process_categories(x):
    if not isinstance(x, (list, np.ndarray)):
        return x
    x = pd.Series(x)
    if 'Não informado' in x:
        x.drop('Não informado', inplace = True)
    if len(x) == 1:
        return x[0]
    return ','.join(sorted(x.astype(str)))

for c in multivalued_categories:
    agg_categorical[c] = agg_categorical[c].apply(
        post_process_categories) 

**O mesmo processo foi efetuado para as variáveis que representam data e target, segue a célula que o fez:**

In [ ]:
agg_target = data.groupby('record_id')['output_os'].last()

In [ ]:
datetime_list = list(map(lambda x: list(x.keys())[0], datetime))
agg_datetime = data.groupby('record_id')[datetime_list].last()

In [ ]:
agg_datetime

In [ ]:
agg_categorical.columns

In [ ]:
agg_categorical

,abortion,alcohol,alcohol_type___1,alcohol_type___2,alcohol_type___3,antec_fam_cancer_mama,breast_feeding,chemotherapy,contraceptive_method___1,contraceptive_method___2,contraceptive_method___3,contraceptive_method___4,contraceptive_method___999,diff_tubular,er_ihc,escolari,familial_degree___1,familial_degree___2,familial_degree___3,family_cancer,family_cancer_id___1,family_cancer_id___2,family_cancer_id___3,family_cancer_id___4,family_cancer_id___5,family_cancer_id___6,grau_hist,her2_fish,her2_ihc,histological_grade,hormone_therapy,mitotic_index,nuclear_grade,period,pr_ihc,pregnancy_history,primary_diganosis,race,rec01,rec02,rec03,rec04,recdist,recidive,reclocal,recregio,tipotrat,tobaco,topo,trathosp,treatment,tobaco_type,meta03,meta02,birads,t,redcap_repeat_instrument,meta04,morfo,n,tumor_stage,sister_tumor_1,mother_tumor,benign,tumor_subtype,ultinfo,tumor_margin,daughter_tumor_1,meta01,m
record_id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
54,Não informado,2,0,0,0,Não informado,Não informado,Não informado,0,0,0,0,0,2,1,2,1,0,0,1,0,0,1,0,0,0,Não informado,5,1,3,Não informado,3,3,Não informado,1,1,2,Não informado,50,Não informado,Não informado,Não informado,0,1,0,0,Não informado,2,445,Não informado,Não informado,1,Não informado,Não informado,5,"11,99","dados_antropometricos,dados_histopatologicos_m...",Não informado,"85003,88211","0,99","10,99",Não informado,Não informado,2,"1,2",2,1,Não informado,Não informado,"0,99"
302,Não informado,Não informado,0,0,0,Não informado,Não informado,Não informado,0,0,0,0,0,Não informado,Não informado,2,0,0,0,Não informado,0,0,0,0,0,0,Não informado,Não informado,4,Não informado,Não informado,Não informado,Não informado,Não informado,Não informado,Não informado,Não informado,Não informado,Não informado,Não informado,Não informado,Não informado,0,0,0,0,Não informado,Não informado,504,Não informado,Não informado,Não informado,Não informado,Não informado,Não informado,2,"dados_antropometricos,dados_histopatologicos_m...",Não informado,85003,0,21,Não informado,Não informado,Não informado,3,4,Não informado,Não informado,Não informado,0
710,Não informado,Não informado,0,0,0,Não informado,Não informado,Não informado,0,0,0,0,0,Não informado,1,4,0,0,0,Não informado,0,0,0,0,0,0,3,Não informado,1,Não informado,Não informado,Não informado,3,Não informado,0,Não informado,Não informado,Não informado,Não informado,Não informado,Não informado,Não informado,0,1,1,1,Não informado,Não informado,508,Não informado,Não informado,Não informado,Não informado,Não informado,Não informado,3,"dados_antropometricos,dados_histopatologicos_m...",Não informado,85003,1,31,Não informado,Não informado,2,2,2,Não informado,Não informado,Não informado,0
752,Não informado,Não informado,0,0,0,Não informado,Não informado,Não informado,0,0,0,0,0,Não informado,Não informado,2,0,0,0,Não informado,0,0,0,0,0,0,Não informado,Não informado,4,Não informado,Não informado,Não informado,Não informado,Não informado,Não informado,Não informado,Não informado,Não informado,Não informado,Não informado,Não informado,Não informado,0,0,0,0,Não informado,Não informado,509,Não informado,Não informado,Não informado,Não informado,Não informado,Não informado,2,"dados_antropometricos,dados_histopatologicos_m...",Não informado,84803,0,21,Não informado,Não informado,Não informado,3,2,Não informado,Não informado,Não informado,0
1589,Não informado,Não informado,0,0,0,Não informado,Não informado,Não informado,0,0,0,0,0,Não informado,1,3,0,0,0,Não informado,0,0,0,0,0,0,3,Não informado,4,Não informado,Não informado,Não informado,Não informado,Não informado,1,Não informado,2,Não informado,Não informado,Não informado,Não informado,Não informado,0,0,0,0,Não informado,Não informado,508,Não informado,Não informado,Não informado,Não informado,Não informado,Não informado,2,"dados_antropometricos,dados_histopatologicos_m...",Não informado,85003,1,22,Não informado,Não informado,2,3,2,Não informado,Não informado,Não informado,0
...,...,...,...,...,.

## Escrever dados

Por fim, agrupamos todos os conjuntos (categóricas, numéricas, alvo, identificadores e variáveis para data) para que fosse possível a análise de cada paciente com uma linha de registro que resuma todas as demais, o resultado final possui 85 colunas tratadas, limpas e agregadas da melhor forma possível, dado o entendimento do negócio no período em que se passava.

In [ ]:
output = pd.merge(
    agg_numerical,
    agg_categorical,
    left_index = True,
    right_index = True,
    how='left')
output = pd.merge(
    output,
    agg_target,
    left_index=True,
    right_index=True,
    how='left')
output = pd.merge(
    output,
    agg_datetime,
    left_index=True,
    right_index=True,
    how='left')

In [ ]:
output

,bmi_median,bmi_std,bmi_min,bmi_max,sister_n,ki67_perc_mean,follow_up_days,follow_up_days_recidive,follow_up_days_recidive_std,height,weight_mean,weight_std,abortion,alcohol,alcohol_type___1,alcohol_type___2,alcohol_type___3,antec_fam_cancer_mama,breast_feeding,chemotherapy,contraceptive_method___1,contraceptive_method___2,contraceptive_method___3,contraceptive_method___4,contraceptive_method___999,diff_tubular,er_ihc,escolari,familial_degree___1,familial_degree___2,familial_degree___3,family_cancer,family_cancer_id___1,family_cancer_id___2,family_cancer_id___3,family_cancer_id___4,family_cancer_id___5,family_cancer_id___6,grau_hist,her2_fish,her2_ihc,histological_grade,hormone_therapy,mitotic_index,nuclear_grade,period,pr_ihc,pregnancy_history,primary_diganosis,race,rec01,rec02,rec03,rec04,recdist,recidive,reclocal,recregio,tipotrat,tobaco,topo,trathosp,treatment,tobaco_type,meta03,meta02,birads,t,redcap_repeat_instrument,meta04,morfo,n,tumor_stage,sister_tumor_1,mother_tumor,benign,tumor_subtype,ultinfo,tumor_margin,daughter_tumor_1,meta01,m,output_os,dob,data_2
record_id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
54,29.15,0.435775,27.7,29.8,-1.0,19.6,3585.0,1338.0,0.0,152.100000,67.335000,1.014591,Não informado,2,0,0,0,Não informado,Não informado,Não informado,0,0,0,0,0,2,1,2,1,0,0,1,0,0,1,0,0,0,Não informado,5,1,3,Não informado,3,3,Não informado,1,1,2,Não informado,50,Não informado,Não informado,Não informado,0,1,0,0,Não informado,2,445,Não informado,Não informado,1,Não informado,Não informado,5,"11,99","dados_antropometricos,dados_histopatologicos_m...",Não informado,"85003,88211","0,99","10,99",Não informado,Não informado,2,"1,2",2,1,Não informado,Não informado,"0,99",1,0,2012-01-15
302,24.80,6.650833,22.4,45.0,-1.0,-1.0,2225.0,-1.0,0.0,157.909091,59.916000,2.921812,Não informado,Não informado,0,0,0,Não informado,Não informado,Não informado,0,0,0,0,0,Não informado,Não informado,2,0,0,0,Não informado,0,0,0,0,0,0,Não informado,Não informado,4,Não informado,Não informado,Não informado,Não informado,Não informado,Não informado,Não informado,Não informado,Não informado,Não informado,Não informado,Não informado,Não informado,0,0,0,0,Não informado,Não informado,504,Não informado,Não informado,Não informado,Não informado,Não informado,Não informado,2,"dados_antropometricos,dados_histopatologicos_m...",Não informado,85003,0,21,Não informado,Não informado,Não informado,3,4,Não informado,Não informado,Não informado,0,1,0,2013-01-17
710,26.40,5.330967,0.0,45.0,-1.0,20.0,3294.0,2442.0,0.0,155.117647,63.451471,1.348497,Não informado,Não informado,0,0,0,Não informado,Não informado,Não informado,0,0,0,0,0,Não informado,1,4,0,0,0,Não informado,0,0,0,0,0,0,3,Não informado,1,Não informado,Não informado,Não informado,3,Não informado,0,Não informado,Não informado,Não informado,Não informado,Não informado,Não informado,Não informado,0,1,1,1,Não informado,Não informado,508,Não informado,Não informado,Não informado,Não informado,Não informado,Não informado,3,"dados_antropometricos,dados_histopatologicos_m...",Não informado,85003,1,31,Não informado,Não informado,2,2,2,Não informado,Não informado,Não informado,0,1,0,2012-08-18
752,35.90,0.525259,35.1,37.2,-1.0,-1.0,4153.0,-1.0,0.0,152.000000,83.110000,1.198923,Não informado,Não informado,0,0,0,Não informado,Não informado,Não informado,0,0,0,0,0,Não informado,Não informado,2,0,0,0,Não informado,0,0,0,0,0,0,Não informado,Não informado,4,Não informado,Não informado,Não informado,Não informado,Não informado,Não informado,Não informado,Não informado,Não informado,Não informado,Não informado,Não informado,Não informado,0,0,0,0,Não informado,Não informado,509,Não informado,Não informado,Não informado,Não informado,Não informado,Não informado,2,"dados_antropometricos,dados_histopatologicos_m...",Não informado,84803,0,21,Não informado,Não informado,Não informado,3,2,Não informado,Não informado,Não informado,0,1,0,2012-06-13
1589,22.90,4.715171,22.2,45.0,-1.0,-1.0,

In [ ]:
pd.set_option('display.max_columns', None)

In [ ]:
output.reset_index().to_csv('/content/drive/MyDrive/MedicinIA/Sample_Data/Processed/DataSet2.csv', index=False)